#Import Python Packages

In [3]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

#Make blank file-info sheet

Get files from database

In [10]:
files = dp.get_metatlas_files(experiment = '%hedlund%',name = '%hilic%',most_recent = True)

Save the files to an empty fileinfo sheet

In [9]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/hedlund_empty_fileinfo.tab',files)

#Create metatlas groups from filled in file-info sheet

In [18]:
dp = reload(dp)
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/FileInfoSheet_BHedlund_AllFiles_20160413 (1).txt',store=False)

#Create a new Atlas in metatlas from a spreadsheet

In [71]:
import pandas as pd

In [93]:
path = '/global/homes/b/bpb/Downloads/hedlund/'
input_file = 'BrianHedlund_Atlas_POS_using_MSMS.csv'
df = pd.read_csv(path + input_file)
df.loc[df.name == 'Arsenobetaine','name'] = ''
output_file = 'Not_Found_' + input_file
print path+output_file
df[df.new_observed == 0][['name','label','rt_min','rt_max','rt_peak','mz','mz_tolerance']].to_csv(path+output_file)

output_file = 'RT_and_MSMS_' + input_file
print path+output_file
df[df.new_observed == 1][['name','label','rt_min','rt_max','rt_peak','mz','mz_tolerance']].to_csv(path+output_file)

output_file = 'RT_' + input_file
print path+output_file
df[df.new_observed == 2][['name','label','rt_min','rt_max','rt_peak','mz','mz_tolerance']].to_csv(path+output_file)

/global/homes/b/bpb/Downloads/hedlund/Not_Found_BrianHedlund_Atlas_POS_using_MSMS.csv
/global/homes/b/bpb/Downloads/hedlund/RT_and_MSMS_BrianHedlund_Atlas_POS_using_MSMS.csv
/global/homes/b/bpb/Downloads/hedlund/RT_BrianHedlund_Atlas_POS_using_MSMS.csv


In [94]:
dp = reload(dp)
# df3 = dp.get_formatted_atlas_from_google_sheet(polarity='POS',method='QE_HILIC', mz_tolerance=10)
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/Not_Found_BrianHedlund_Atlas_POS_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_pos_not observed',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=True,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_and_MSMS_BrianHedlund_Atlas_POS_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_pos_present',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=True,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_BrianHedlund_Atlas_POS_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_pos_maybe',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=True,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/Not_Found_BrianHedlund_Atlas_NEG_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_neg_not observed',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'negative',
                                       store=True,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_and_MSMS_BrianHedlund_Atlas_NEG_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_neg_present',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'negative',
                                       store=True,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/hedlund/RT_BrianHedlund_Atlas_NEG_using_MSMS.csv',
                                       'hedlund_hilic_qexactive_neg_maybe',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'negative',
                                       store=True,
#                                        dataframe = df3,
                                      mz_tolerance = 20)

In [4]:
project_label

NameError: name 'project_label' is not defined

In [ ]:
atlases = ['hedlund_hilic_qexactive_neg_not observed',
        'hedlund_hilic_qexactive_neg_present',
        'hedlund_hilic_qexactive_neg_maybe',
        'hedlund_hilic_qexactive_pos_not observed',
        'hedlund_hilic_qexactive_pos_maybe',
        'hedlund_hilic_qexactive_pos_present']


import os           

for a in atlases:
#     if 'neg' in a:
#         group_str = '%hedlund%neg%'
#     else:
#         group_str = '%hedlund%pos%'
#     groups = dp.select_groups_for_analysis(name = group_str, most_recent = True, remove_empty = True, filter_list = ['Set1'], exclude_list = ['QC','Blank'])
        
#     atlas = dp.get_metatlas_atlas(name=a,do_print = True)
    
#     TODO: Export the searched data instead of the atlas
#     df = dp.export_atlas_to_spreadsheet(atlas[0],'/global/homes/b/bpb/Downloads/hedlund/set1/%s.csv'%a)
    output_filename = '/global/homes/b/bpb/Downloads/hedlund/set1/pickle_%s.pkl'%a
    print output_filename
#     data = dp.get_data_for_groups_and_atlas(groups,atlas[0],output_filename,use_set1 = True)
    
    my_str = os.path.basename(output_filename).split('.')[0].replace(' ','_').replace('pickle_hedlund_','')
    project_label = '/global/homes/b/bpb/Downloads/hedlund/set1/%s'%my_str
    print project_label
    peak_height = dp.make_output_dataframe(input_fname = output_filename, fieldname='peak_height' , output_loc=project_label+'/sheets')
#     peak_area = dp.make_output_dataframe(input_fname = output_filename, fieldname='peak_area' , output_loc=project_label+'/sheets')
#     rt_peak = dp.make_output_dataframe(input_fname = output_filename, fieldname='rt_peak' , output_loc=project_label+'/sheets')
#     mz_centroid = dp.make_output_dataframe(input_fname = output_filename, fieldname='mz_centroid' , output_loc=project_label+'/sheets')

    dp.plot_errorbar_plots(peak_height,input_fname = output_filename, output_loc=project_label+'/error_bar_peak_height')
#     dp.plot_all_files_for_each_compound(input_fname = output_filename, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')
#     dp.make_identification_figure(input_fname = output_filename, output_loc=project_label+'/identification')
    
    


/global/homes/b/bpb/Downloads/hedlund/set1/pickle_hedlund_hilic_qexactive_neg_not observed.pkl
/global/homes/b/bpb/Downloads/hedlund/set1/hilic_qexactive_neg_not_observed
/global/homes/b/bpb/Downloads/hedlund/set1/pickle_hedlund_hilic_qexactive_neg_present.pkl
/global/homes/b/bpb/Downloads/hedlund/set1/hilic_qexactive_neg_present
/global/homes/b/bpb/Downloads/hedlund/set1/pickle_hedlund_hilic_qexactive_neg_maybe.pkl
/global/homes/b/bpb/Downloads/hedlund/set1/hilic_qexactive_neg_maybe
/global/homes/b/bpb/Downloads/hedlund/set1/pickle_hedlund_hilic_qexactive_pos_not observed.pkl
/global/homes/b/bpb/Downloads/hedlund/set1/hilic_qexactive_pos_not_observed
/global/homes/b/bpb/Downloads/hedlund/set1/pickle_hedlund_hilic_qexactive_pos_maybe.pkl
/global/homes/b/bpb/Downloads/hedlund/set1/hilic_qexactive_pos_maybe
/global/homes/b/bpb/Downloads/hedlund/set1/pickle_hedlund_hilic_qexactive_pos_present.pkl

#Select groups of files to operate on

In [53]:

dp = reload(dp)

groups = dp.select_groups_for_analysis(name = '%hedlund%neg%',most_recent = True, remove_empty = True, filter_list = [], exclude_list = ['QC','Blank'])
# for g in groups:
#     print g.name

0 20160413_Bhedlund_pHILIC_NEG_GBS_Med_inc_Set1 2016-04-14 17:59:30
1 20160413_Bhedlund_pHILIC_NEG_PR_End_Set1 2016-04-14 17:59:36
2 20160413_Bhedlund_pHILIC_NEG_JKG1_PR_Set1 2016-04-14 17:59:33
3 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_Ctrl_Set1 2016-04-14 17:59:32
4 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_4OE_Set1 2016-04-14 17:59:37
5 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_inc_Set2 2016-04-13 22:34:26
6 20160413_Bhedlund_pHILIC_NEG_G233_BMR_Set1 2016-04-14 17:59:19
7 20160413_Bhedlund_pHILIC_NEG_BMR_End_Set1 2016-04-14 17:59:39
8 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_32OE_Set2 2016-04-13 22:34:27
9 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_32OE_Set1 2016-04-14 17:59:20
10 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_Ctrl_Set2 2016-04-13 22:34:26
11 20160413_Bhedlund_pHILIC_NEG_GBS_Med_Ctrl_Set1 2016-04-14 17:59:27
12 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_Set1 2016-04-14 17:59:39
13 20160413_Bhedlund_pHILIC_NEG_PR_Beg_Set1 2016-04-14 17:59:28
14 20160413_Bhedlund_pHILIC_NEG_GBS_4OE_Ctrl_Set1 2016-04

#Select Atlas to use

In [6]:
dp = reload(dp)
atlas = dp.get_metatlas_atlas(name='%hedlund_hilic_qexactive_pos%',do_print = True)


0 15 hedlund_hilic_qexactive_pos_maybe 2016-05-05 23:40:23
1 40 hedlund_hilic_qexactive_pos 2016-04-27 16:02:59
2 48 hedlund_hilic_qexactive_pos_not observed 2016-05-05 23:40:17
3 23 hedlund_hilic_qexactive_pos_present 2016-05-05 23:40:21


In [67]:
myAtlas = atlas[1]
myAtlas.name

u'hedlund_hilic_qexactive_neg_not observed'

#Export Atlas To Spreadsheet

In [68]:
dp = reload(dp)
df = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/Downloads/hedlund_hilic_qexactive_neg_not_observed.csv')
df.head()

,name,inchi,mono_isotopic_molecular_weight,creation_time,formula,last_modified,unique_id,username,label,rt_min,...,common_name,synonyms,pubchem_compound_id,lipidmaps_id,metacyc_id,hmdb_id,img_abc_id,chebi_id,kegg_id,metatlas_inchi
0,2-<i>trans</i>-abscisate,InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)1...,264.136169,1444322247,C15H20O4,1444322293,75359ff3b38c45bb9e87cf292349eaeb,bpb,abscisic acid (peak 2),1.77,...,2-trans-abscisate///(+)-abscisate///(R)-2-tran...,2-trans-abscisic acid anion///abscisic acid///...,7251168///6951345,nan,CPD-693///CPD-7731,nan,nan,CHEBI:37569///CHEBI:62429///CHEBI:62421///CHEB...,C06082///C06082,InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)1...
1,aminoadipic acid,InChI=1S/C6H11NO4/c7-4(6(10)11)2-1-3-5(8)9/h4H...,161.068808,1448915919,C6H11NO4,1448915929,587bcf3788494ca9a457ef828b376862,bpb,aminoadipic acid,13.75,...,L-2-aminoadipate(1-)///L-2-aminoadipate///2-am...,DL-alpha-Aminoadipic acid///DL-2-Aminoadipate/...,6994046///469///6992111,LMFA01170098,CPD-468///CPD-9400,HMDB00510,nan,CHEBI:37024///CHEBI:84981///CHEBI:17082///CHEB...,C00956,InChI=1S/C6H11NO4/c7-4(6(10)11)2-1-3-5(8)9/h4H...
2,biotin,InChI=1S/C10H16N2O3S/c13-8(14)4-2-1-3-7-9-6(5-...,244.088165,1444322230,C10H16N2O3S,1444322292,442ba7da0b66469b87c8e0a9978bb689,bpb,biotin,6.83,...,"biotinate///biotin///5-(2-oxo-1,3,3a,4,6,6a-he...",d-biotin///vitamin H///nan///coenzyme R///biot...,171548.0///6560210,nan,BIOTIN,nan,nan,CHEBI:95156///CHEBI:57586,C00120,InChI=1S/C10H16N2O3S/c13-8(14)4-2-1-3-7-9-6(5-...
3,carnosine,InChI=1S/C9H14N4O3/c10-2-1-8(14)13-7(9(15)16)3...,226.106598,1444322253,C9H14N4O3,1444322294,0da1ed67166e4c18b8c95addd7c02b85,bpb,carnosine,13.61,...,carnosine///carnosine zwitterion///Carnosine,N-(3-Aminopropanoyl)histidine///Carnosine///Ka...,6992100,nan,CARNOSINE,HMDB00033,nan,CHEBI:15727///CHEBI:57485,C00386,InChI=1S/C9H14N4O3/c10-2-1-8(14)13-7(9(15)16)3...
4,L-citrulline,InChI=1S/C6H13N3O3/c7-4(5(10)11)2-1-3-9-6(8)12...,175.095688,1444322245,C6H13N3O3,1444322293,35d82f426ffd42cd99a2e2e35ea8f583,bpb,citrulline,12.81,...,Citrulline///L-citrulline zwitterion///L-citru...,N(delta)-Carbamylornithine///Ngamma-carbamylor...,6992098,nan,L-CITRULLINE,HMDB00904,65567,CHEBI:57743///CHEBI:16349,C00327,InChI=1S/C6H13N3O3/c7-4(5(10)11)2-1-3-9-6(8)12...


#Get Data and Save it

In [69]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
dp = reload(dp)
output_filename = '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Not_Observed.pkl'
data = dp.get_data_for_groups_and_atlas(groups,myAtlas,output_filename,use_set1 = True)

0 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH36.h5
0 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH37.h5
0 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH38.h5
0 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH39.h5
0 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH40.h5
1 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH6.h5
1 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH7.h5
1 17 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150

In [70]:
# my_files = ['/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Present.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Maybe.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Not_Observed.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Present.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Maybe.pkl',
#            '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Not_Observed.pkl']
my_files = ['/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_NEG_Not_Observed.pkl']
import os           
for my_file in my_files:
    project_label = '/global/homes/b/bpb/Downloads/hedlund/%s'%os.path.basename(my_file)
    peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')
    dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')
    dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=True , output_loc=project_label+'/scaled_all_compounds_each_file')
    dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=True , output_loc=project_label+'/scaled_all_files_each_compound')
    dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=False , output_loc=project_label+'/all_compounds_each_file')
    dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')
    dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

3
24
nrows =  13
length of ymax is  1493
3
24
nrows =  13
length of ymax is  1493
1.42891201477
0.0306058562391
1.84533379346
0.0829445487586
0.0442709141713
0.0953678457134
27.9690249922
10.4045970154
0.0803545640557


#Load a pre-existing datafile

In [45]:
output_filename = '/global/homes/b/bpb/data/hedlund/20160503_Bhedlund_pHILIC_POS_Present.pkl'
my_file = output_filename #'/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'

project_label = '/global/homes/b/bpb/Downloads/hedlund/TestIDFigs'
data = dp.get_data(my_file)

In [51]:
data[0][2]['identification']

{'compound': [{'creation_time': '2015-10-08T17:36:23',
 'description': u'',
 'formula': u'C9H18O8',
 'functional_sets': [],
 'head_id': u'bb965dc128f04ea8a8bc426b80b31627',
 'inchi': u'InChI=1S/C9H18O8/c10-1-4(2-11)16-9-8(15)7(14)6(13)5(3-12)17-9/h4-15H,1-3H2/t5-,6-,7+,8-,9+/m1/s1',
 'inchi_key': None,
 'last_modified': '2015-10-08T17:38:11',
 'mono_isotopic_molecular_weight': 254.1001739501953,
 'name': u'2-(&alpha;-D-glucopyranosyl)-<i>sn</i>-glycerol',
 'neutralized_2d_inchi': None,
 'neutralized_2d_inchi_key': None,
 'neutralized_inchi': None,
 'neutralized_inchi_key': None,
 'number_components': None,
 'permanent_charge': None,
 'prev_uid': u'origin',
 'reference_xrefs': [],
 'synonyms': u'',
 'unique_id': u'bb965dc128f04ea8a8bc426b80b31627',
 'url': u'',
 'username': u'bpb'}],
 'creation_time': '2016-05-03T21:37:56',
 'description': u'No description',
 'frag_references': [],
 'head_id': u'04afa82cfada434b90010989861ea36b',
 'identification_grade': None,
 'last_modified': '2016-05

#Make Supplementary Tables

In [14]:
peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [15]:
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [16]:
dp = reload(dp)
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=True , output_loc=project_label+'/scaled_all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=True , output_loc=project_label+'/scaled_all_files_each_compound')
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=False , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')

5
40
nrows =  13
length of ymax is  2803
5
40
nrows =  13
length of ymax is  2803


#Make Identification Figures

In [8]:
dp = reload(dp)
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

6.59247657306
21.4827464711
6.84921022986
2.65480287199
6.25471107625
1.5158428629
13.3272408184
5.11620742382
71.8406682781
6.08595159
368.810113072
0.462781928553
503.98781674
7.43189986404
11.5629219351
8.78786173883
1.22372570539
19.1785529618
3.33191714632
602.605887609
4.42926455442
0.220687110029
0.665087360364
0.0134621266239
139.518487497
25.1960069603
31.7161324889
56.0450795641
3.003265765
